In [ ]:
# this was used in doc printing
#\tableofcontents \newpage

# Käänteismatriisi modulo p¶

This is related to a school project and therefore the documentation is in Finnish.

## Tehtävän kuvaus:
Matriisin kääntäminen modulo p, käyttäen lajennettua Eukleideen algoritmia,

## Suunnitelma tehtävän ratkaisemiseksi

Olkoon käännettävä matriisi A ja p jokin kokonaisluku:

1. Lasketaan detA ja jotta kyseisellä menetelmällä voidaan kääntää matriisi A modulo p, niin varmistetaan, että; 
    * matriisin determinantti on erisuuri kuin 0 ja 
    * matriisin determinantin ja kokonaisluvun p suurin yhteinen tekijä on 1
2. Muodostetaan matriisin A kotekijöiden avulla adjungoitu matriisi adjA
3. Transponoidaan apumatriisi adjA
4. Etsitään determinantille käänteisluku mod p, käyttäen avuksi laajennettua Eukleideen algoritmia
5. Kerrotaan käännetty adjA-matriisi arvolla, joka saadaan yhtälöstä 1/detA mod p
6. Sievennetään käänteismatriisin arvot modulo p

Muuta: Oletuksena on, että käännetävä matriisi on neliömatriisi ja matriisin A alkiot ovat kokonaislukuja.<br>
Huomioitavaa: käytetyn menetelmän sisältävän rekursion vuoksi algoritminen kompleksisuus eksponentiaalinen, joten ratkaisu ei sovellu suurempien matriisien kääntämiseen.

## Funktioiden rakentaminen

### Alimatriisi

Tehdään ensin apufunktio, joka etsii matriisin tietyn kohdan pienemmän matriisin (n−1)×(n−1). Tätä funktiota tarvitaan myöhemmin muissa funktioissa.<br>

Algoritmi: 

1. alimatriisi, parametrit: matriisi a, rivi j, sarake j:
2. uusi matriisi b on matriisin a kopio (pelkkä b=a viittaisin samaan matriisiin, jolloin funktio aiheuttaisi sivuvaikutuksia)
3. poistetaan matriisin b rivi i
4. järjestetään matriisi rivien sijaan väliaikaisesti sarakkeittain, jotta seuraavaksi saadaan j-sarake poistettua (zip yhdistettynä *-operaattoriin, kokoan listan jokaisen alkion ensimmäiset yhteen, toiset yhteen jne -> b:n sarakkeet)
5. poistetaan matriisin b sarake j
6. järjestetään sarakkeiden sijaan riveittäin (zip yhdeistettynä *-operaattoriin listaa jäljellä olevien sarakkeiden ensimmäiset alkiot yhteen jne -> b:n rivit)
7. palautetaan alimatriisi b

In [1]:
def alimatriisi(a,i,j):
    b = a[:] # kopio a:sta, jolloin ei vaikuta a:han
    del(b[i]) # poistetaan rivi i
    b = zip(*b) # järjestetään sarakkeittain
    del(b[j]) # poistetaan sarake j
    b = zip(*b) # järjestetään riveittäin
    return b # palautetaan alimatriisi b

In [2]:
mtr1=[[23, 3],[10,25]]

In [3]:
mtr2=[[2, -1, 0], [1, 0, -5], [4, 3, -9]]

Havainnollisuuden vuoksi otetaan käyttöön numpy kirjasto, jolla matriisia voi tarkastella matriisimuodossa. Tämä ei muuta muuttujien sisältöä.

In [4]:
import numpy as np
print np.matrix(mtr1)
print np.matrix(mtr2)

[[23  3]
 [10 25]]
[[ 2 -1  0]
 [ 1  0 -5]
 [ 4  3 -9]]


Testataan alimatriisi-funktiota

In [5]:
print alimatriisi(mtr1,0,0) # pitäisi olla 25
print alimatriisi(mtr2,0,0) # pitäisi olla [[0, -5], [3, -9]]
print alimatriisi(mtr2,1,2) # pitäisi olla [[2, -1], [4, 3]]

[(25,)]
[(0, -5), (3, -9)]
[(2, -1), (4, 3)]


### Determinantti

Funktio, joka etsii matriisin determinantin kofaktoreiden avulla.<br><br>

$detA=c_{ij}*(-1)^{i+j}$<br><br>

Algoritmi:

1. determinantti, parametrit: matriisi a:
2. kun matriisin pituus on vain yksi: 
3. palautetaan kyseinen luku rekursiiviselle kutsulle
4. muuten
5. matriisin pituus tallennetaan muuttujaan n
6. determinantiksi asetetaan 0, jotta voidaan myöhemmin lisätä siihen rekursiivisen kutsun arvoja
7. käydään matriisia läpi sarakkeiden lukumäärän verran (sarakkeita yhtä paljon kuin rivejä)
8. käydään matriisin ensimmäinen rivi läpi sarakkeittain ja kerrotaan kyseinen arvo kofaktorilla
9. palautetaan determinantti

In [6]:
def determinantti(a):
    if len(a) == 1: # 
        return a[0][0]
    else:
        n=len(a)
        det = 0
        for i in range(n): # käydään rivi 0 läpi sarakkeittain; a[0][i]
            # kutsutaan rekursiivisesti ja lisätään summaan
            det += a[0][i] * (-1)**(i) * determinantti(alimatriisi(a,0,i)) 
        return det #Adds successive elements times their cofactors

In [7]:
'''testaus 1
pitäisi olla 25·23-3·10 = 545'''
print determinantti(mtr1) # 

'''testaus 2
pitäisi olla 2·0·(-9) + (-1)·(-5)·4 + 0·1·3 - 0·0·4 - 2·(-5)·3 - (-1)·1·(-9) 
           = 0 + 20 + 0 - 0 + 30 - 9 = 41'''
print determinantti(mtr2) # 


545
41


In [8]:
def idmatriisi(n):
    # diagonaalilla x=y, jolloin arvo on 1, muulloin 0
    return [[long(x == y) for x in range(0, n)] for y in range(0, n)] 

In [9]:
print np.matrix(idmatriisi(3))

[[1 0 0]
 [0 1 0]
 [0 0 1]]


### Adjungoitu matriisi adjA kotekijöiden avulla

Algoritmi:

1. adjungoitu matriisi, parametrit: matriisi a:
2. luodaan uuden matriisin pohjaksi identiteettimatriisi adja, joka on samankokoinen kuin matriisi a
3. uuteen matriisiin lasketaan a:n avulla alkiot yksi kerrallaan, aikaisempien funktioiden avulla<br>
$(-1)^{rivi+sar}*determinantti(alimatriisi(a,rivi,sar))$<br>
4. palautetaan matriisi adja 

In [10]:
def adjA(a):
    n = len(a)
    adja = idmatriisi(n)
    for rivi in xrange(n):
        for sar in xrange(n):
            adja[rivi][sar] = (-1)**(rivi+sar) * determinantti(alimatriisi(a,rivi,sar))            
    return adja

In [11]:
adjA(mtr1)

[[25, -10], [-3, 23]]

In [12]:
print np.matrix(adjA(mtr2))

[[ 15 -11   3]
 [ -9 -18 -10]
 [  5  10   1]]


### Transponointi

Algoritmi:

1. transponoitu matriisi, parametrit: matriisi a
2. palautetaan matriisi, jossa matriisin a[rivi][sar] rivien arvot on asetettu sarakkeiden arvoiksi, eli transponoitu<br>


In [13]:
def trans(a):
    return [[a[sar][rivi] for sar in range(len(a)) ] for rivi in range(len(a))] 

In [14]:
print np.matrix(mtr1), '\n'
print np.matrix(trans(mtr1))

[[23  3]
 [10 25]] 

[[23 10]
 [ 3 25]]


In [15]:
print np.matrix(mtr2), '\n'
print np.matrix(trans(mtr2))

[[ 2 -1  0]
 [ 1  0 -5]
 [ 4  3 -9]] 

[[ 2  1  4]
 [-1  0  3]
 [ 0 -5 -9]]


### Modulo p
Määritellään funktiot laajennetulle Eukleideen algoritmille ja maanteisluvun laskemiselle mod p.

#### Eukleideen algoritmi

(syt(a,b))

#### Laajennettu Eukleideen algoritmi

arvot $u$ ja $v$; $syt(a, b) = r_{n-1} = u*a + v*b$

##### Esimerkki

Etsitään arvoille a = 65 ja b = 40 suurin yhteinen tekijä, sekä kartoimet u ja v.

<b>Eukleideen algoritmi</b><br>
Etsitään suurin yhteinen tekijä Eukleideen (tavallisella) algoritmilla:<br> 

(1) 65 = 1 · 40 + 25 
(2) 40 = 1 · 25 + 15 
(3) 25 = 1 · 15 + 10 
(4) 15 = 1 · 10 + 5 10 = 2 · 5

Toisin sanoen<br> 
$SYT(65, 40) = 5.$<br><br>
<b>Laajennettu Eukleideen algoritmi</b><br>
Etsitään kertoimet $u$ ja $v$ laskemalla algoritmia takaperin:
$SYT(86,40)= 5$<br>

(4) = 15 − 10 <br>
(3) = 15 − (25 − 15) = 2 · 15 − 25 <br>
(2) = 2 (40−25) − 25 = 2 · 40 − 3 · 25<br>
(1) = 2 · 40−3 (65 − 40)=5 · 40 − 3 · 65<br><br>

Saadaan:<br>
65(−3) + 40(5) = 5<br>

In [16]:
# Suurin yhteinen tekijä, käytetään tarkistuksissa
def syt(a, b):
    if b == 0:
        return a
    else:
        return syt(b, a % b)

In [17]:
def lea(a, b): #laajennettu Eukleideen algoritmi
    if b == 0:
        return (1, 0)
    else:
        (x, y) = lea(b, a % b)
        return (y, x - (a / b) * y)

In [18]:
a = 65
b = 40
print 'lukujen ',a,' ja ',b,'\nsyt: ',syt(a,b),'\nv ja u: ' ,lea(a,b), \
        '\ntarkistus', lea(a,b)[0]*a+ lea(a,b)[1]*b==syt(a,b)

lukujen  65  ja  40 
syt:  5 
v ja u:  (-3, 5) 
tarkistus True


In [19]:
a=1239
b=735
print 'lukujen ',a,' ja ',b,'\nsyt: ',syt(a,b),'\nv ja u: ' ,lea(a,b), \
        '\ntarkistus', lea(a,b)[0]*a+ lea(a,b)[1]*b==syt(a,b)

lukujen  1239  ja  735 
syt:  21 
v ja u:  (-16, 27) 
tarkistus True


In [20]:
b = 18273645
a = 34527 
print 'lukujen ',a,' ja ',b,'\nsyt: ',syt(a,b),'\nv ja u: ' ,lea(a,b), \
        '\ntarkistus', lea(a,b)[0]*a+ lea(a,b)[1]*b==syt(a,b)

lukujen  34527  ja  18273645 
syt:  3 
v ja u:  (-892856, 1687) 
tarkistus True


#### Käänteisluku modulo p

Lasketaan käänteisluku (mod p) käyttäen avuksi aiemmin määriteltyä funktiota lea(), joka laskee 

In [21]:
def invmodp(a, p):
    a = a % p
    if (a == 0):
        return 0
    (x,y) = lea(p, a % p)
    return y % p

In [22]:
print invmodp(121,13)
print 121*invmodp(121,13)%13==1 # tarkistus

10
True


In [23]:
print invmodp((34527),17)
print 34527*invmodp(34527,17)%17==1 # tarkistus

0
False


In [24]:
print invmodp((34527),5875)
print 34527*invmodp(34527,5875)%5875==1 # tarkistus

4713
True


### Käänteismatriisi modulo p

Jos annetun matriisin A determinantti on nolla tai determinantin ja annetun kokonaisluvun p suurin yhteinen tekijä ei ole 1, niin funktio antaa ilmoituksen ongelmasta ja palauttaa käänteismatriisina vain luvun 0.

In [25]:
def invmatmod(a, p):
    deta=determinantti(a)
    n=len(a)
    if deta==0: # determinanttitarkistus 1
        print "Determinantti on 0. Käänteismatriisia ei voi laskea."
        return 0
    elif syt(deta,p)!=1: # determinanttitarkistus 2
        print "Matriisin A determinantilla ja arvolla p on yhteisiä tekijöitä."
        print "Käänteismatriisia ei voi laskea."
        return 0
    else:
        komat=adjA(a)
        kaant_komat=trans(komat)
        detmod=invmodp(deta,p)
        ainvmod=[[(detmod*kaant_komat[rivi][sar])%p for sar in range(n) ]\
                 for rivi in range(n)]
        return ainvmod

In [26]:
np.matrix(invmatmod(mtr1,26))

matrix([[ 1,  3],
        [10,  3]])

In [27]:
# tarkistus
# käytetään tässä numpy-moduulin matriisilaskua hyödyksi
np.matrix(mtr1)*np.matrix(invmatmod(mtr1,26))%26

matrix([[1, 0],
        [0, 1]])

In [28]:
np.matrix(invmatmod(mtr2,26))

matrix([[ 1, 15,  9],
        [ 1,  4, 18],
        [21,  8,  7]])

In [29]:
# tarkistus

np.matrix(mtr2)*np.matrix(invmatmod(mtr2,26))%26

matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

Testataan vielä determinanttitarkistusten toiminta

In [30]:
print "Determinantti: ", determinantti([[2, -10], [-3, 15]])
np.matrix(invmatmod([[2, -10], [-3, 15]],26))

Determinantti:  0
Determinantti on 0. Käänteismatriisia ei voi laskea.


matrix([[0]])

In [31]:
print "Determinantti: ", determinantti([[2, -10], [-3, 16]])
np.matrix(invmatmod([[2, -10], [-3, 16]],26))

Determinantti:  2
Matriisin A determinantilla ja arvolla p on yhteisiä tekijöitä.
Käänteismatriisia ei voi laskea.


matrix([[0]])

## Funktiot koottuna yhteen ja lopullinen testaus

In [32]:
def alimatriisi(a,i,j):
    b = a[:] # kopio a:sta, jolloin ei vaikuta a:han
    del(b[i]) # poistetaan rivi i
    b = zip(*b) # järjestetään sarakkeittain
    del(b[j]) # poistetaan sarake j
    b = zip(*b) # järjestetään riveittäin
    return b # palautetaan alimatriisi b

def determinantti(a):
    if len(a) == 1: # 
        return a[0][0]
    else:
        n=len(a)
        det = 0
        for i in range(n): # käydään rivi 0 läpi sarakkeittain; a[0][i]
            # kutsutaan rekursiivisesti ja lisätään summaan
            det += a[0][i] * (-1)**(i) * determinantti(alimatriisi(a,0,i)) 
        return det #Adds successive elements times their cofactors

def idmatriisi(n):
    # diagonaalilla x=y, jolloin arvo on 1, muulloin 0
    return [[long(x == y) for x in range(0, n)] for y in range(0, n)] 

def adjA(a):
    n = len(a)
    adja = idmatriisi(n)
    for rivi in xrange(n):
        for sar in xrange(n):
            adja[rivi][sar] = (-1)**(rivi+sar) * determinantti(alimatriisi(a,rivi,sar))            
    return adja

def lea(a, b): #laajennettu Eukleideen algoritmi
    if b == 0:
        return (1, 0)
    else:
        (x, y) = lea(b, a % b)
        return (y, x - (a / b) * y)

def invmodp(a, p):
    a = a % p
    if (a == 0):
        return 0
    (x,y) = lea(p, a % p)
    return y % p

def invmatmod(a, p):
    deta=determinantti(a)
    n=len(a)
    if deta==0: # determinanttitarkistus 1
        print "Determinantti on 0. Käänteismatriisia ei voi laskea."
        return 0
    elif syt(deta,p)!=1: # determinanttitarkistus 2
        print "Matriisin A determinantilla ja arvolla p on yhteisiä tekijöitä."
        print "Käänteismatriisia ei voi laskea."
        return 0
    else:
        komat=adjA(a)
        kaant_komat=trans(komat)
        detmod=invmodp(deta,p)
        ainvmod=[[(detmod*kaant_komat[rivi][sar])%p for sar in range(n) ] \
                 for rivi in range(n)]
        return ainvmod

Lasketaan esimerkkimatriisien käänteismatriiseja modulo p ja tehdään lisäksi taskistus laskemalla $AxA^{-1}(\mod p)$. Käytetään tässä apuna numpy-kirjaston matriisilaskentaa (np.matrix).

### Esimerkkejä

#### Esimerkki 1

Matriisi 
$A = 
 \begin{pmatrix}
  5 & 8 \\
  8 & 3 
 \end{pmatrix}$ ja $p=26$

In [33]:
matriisi1=[[5,8],[8,3]]
np.matrix(invmatmod(matriisi1,26))

matrix([[ 1,  6],
        [ 6, 19]])

Saadaan $A^{-1} = 
 \begin{pmatrix}
  1 & 6 \\
  6 & 19 
 \end{pmatrix}(\mod 26)$<br><br>
 Lasketaan vielä tarkistus,
 $AxA^{-1}(\mod 26)$, jonka tulos pitäisi olla identiteettimatriisi $I$

In [34]:
# tarkistus
np.matrix(matriisi1)*np.matrix(invmatmod(matriisi1,26))%26

matrix([[1, 0],
        [0, 1]])

#### Esimerkki 2

$A =
 \begin{pmatrix}
  6 & 24 & 1 \\
  13 & 16 & 10 \\
  20 & 17 & 15
 \end{pmatrix}$ ja $p=26$

In [35]:
matriisi2=[[6, 24, 1], [13, 16, 10], [20, 17, 15]]
np.matrix(invmatmod(matriisi2,26))

matrix([[ 8,  5, 10],
        [21,  8, 21],
        [21, 12,  8]])

Saadaan $A^{-1} = 
 \begin{pmatrix}
  8 & 5 & 10 \\
  21 & 8 & 21 \\
  21 & 12 & 8
 \end{pmatrix}(\mod 26)$ <br><br>
 Lasketaan vielä tarkistus,
 $AxA^{-1}(\mod 26)$, jonka tulos pitäisi olla identiteettimatriisi $I$

In [36]:
# Lasketaan vielä tarkistus,
#  $AxA^{-1}(\mod 26)$, jonka tulos pitäisi olla identiteettimatriisi $I$

In [37]:
np.matrix(matriisi2)*np.matrix(invmatmod(matriisi2,26))%26

matrix([[1, 0, 0],
        [0, 1, 0],
        [0, 0, 1]])

#### Esimerkki 3

$A = 
 \begin{pmatrix}
  1 & 0 & 9 & 0 \\
  0 & 2 & 3 & 0 \\
  3 & 4 & 0 & 1 \\
  10 & 1 & 2 & 1 \\
 \end{pmatrix}$ ja $p=13$

In [38]:
matriisi3=[[1, 0, 9, 0], [0, 2, 3, 0], [3, 4, 0, 1], [10, 1, 2, 1]]
np.matrix(invmatmod(matriisi3,13))

matrix([[ 0,  3, 11,  2],
        [ 2,  1,  4,  9],
        [ 3,  4,  6,  7],
        [ 5,  0,  4, 10]])

Saadaan $A^{-1} = 
 \begin{pmatrix}
  0 & 3 & 11 & 2 \\
  2 & 1 & 4 & 9 \\
  3 & 4 & 6 & 7 \\
  5 & 0 & 4 & 10
 \end{pmatrix}(\mod 26)$<br><br>
 Lasketaan vielä tarkistus,
 $AxA^{-1}(\mod 26)$, jonka tulos pitäisi olla identiteettimatriisi $I$

In [39]:
np.matrix(matriisi3)*np.matrix(invmatmod(matriisi3,13))%13

matrix([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]])

#### Esimerkki 4

$A = 
 \begin{pmatrix}
  1 & 2 & 3 \\
  4 & 5 & 6 \\
  7 & 8 & 9
 \end{pmatrix}$ ja $p=7$,<br>
 jossa matriisin A determinantti on 0, eikä käänteismatriisia voi laskea.

In [40]:
matriisi4=[[1,2,3],[4,5,6],[7,8,9]]
np.matrix(invmatmod(matriisi4,7))

Determinantti on 0. Käänteismatriisia ei voi laskea.


matrix([[0]])

#### Esimerkki 5 a)

Matriisi 
$A = 
 \begin{pmatrix}
  1 & 3 \\
  1 & 5 
 \end{pmatrix}$ ja $p=26$,<br>jossa matriisin A determinantti ja kokonaisluku p ovat molemmat kahdella jaollisia, joten kyseistä algoritmia ei voi käyttää matriisin A kääntämiseen modulo p.

In [41]:
matriisi5=[[1, 3], [1, 5]]
np.matrix(invmatmod(matriisi5,26))

Matriisin A determinantilla ja arvolla p on yhteisiä tekijöitä.
Käänteismatriisia ei voi laskea.


matrix([[0]])

#### Esimerkki 5 b)

Matriisi 
$A = 
 \begin{pmatrix}
  1 & 3 \\
  1 & 5 
 \end{pmatrix}$ ja $p=7$    

In [42]:
np.matrix(invmatmod(matriisi5,7))

matrix([[6, 2],
        [3, 4]])

Saadaan $A^{-1} = 
 \begin{pmatrix}
  6 & 2 \\
  3 & 4 
 \end{pmatrix}(\mod 7)$<br><br>
 Lasketaan vielä tarkistus,
 $AxA^{-1}(\mod 7)$, jonka tulos pitäisi olla identiteettimatriisi $I$

In [43]:
np.matrix(matriisi5)*np.matrix(invmatmod(matriisi5,7))%7

matrix([[1, 0],
        [0, 1]])